In [188]:
# Loading data

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
DF=pd.concat([train,test])

In [189]:
# Decoding names
def decodeTitle(row):
    signif_titles=['Mr.','Miss.','Mrs.','Master.'] 
    change_sex={'female':'Miss.','male':'Mr.'}
    # cut exactly title from column Name
    title=row[4][row[4].find(',')+2:row[4].find('.')+1]
    
    
    # return title if it is in set of significant titles
    if title in signif_titles:
        return(title)
    
    # return sex if title is insignificant
    else: return(change_sex[row[8]])
    
DF['Title']=DF.apply(decodeTitle, axis=1)
DF['Fulltitle']=DF.apply(lambda x: x[4][x[4].find(',')+2:x[4].find('.')+1], axis=1)

In [190]:
# Decodng cabins
def number(string):
    if pd.notnull(string):
        numbers=[int(i) for i in re.findall('(\d+)',string)]
        return(np.mean(numbers))

DF['CabinSymbol'] = DF.Cabin.str[0].fillna('X')
DF['CabinNumber'] = DF.Cabin.apply(number)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [191]:
Ages, Fares=[0],[0]
for i in [1,2,3]:
    Ages.append(DF[DF.Pclass==i]['Age'].mean())
    Fares.append(DF[DF.Pclass==i]['Fare'].mean())
def funca(row):
    if row[0]>0: 
        age=row[0]
    else: 
        age=Ages[row[7]]
    return(age)
def funcb(row):
    if row[3]>0: 
        fare=row[3]
    else: 
        fare=Fares[row[7]]
    return(fare)
                 
DF['Age']=DF.apply(funca,axis=1)
DF['Fare']=DF.apply(funcb,axis=1)

In [192]:
DF['Title']=DF['Title'].astype('category').cat.codes
DF['CabinSymbol']=DF['CabinSymbol'].astype('category').cat.codes
DF['Sex']=DF['Sex'].astype('category').cat.codes

In [199]:
DF_train=DF[DF.Survived.notnull()]
DF_test=DF[DF.Survived.notnull()==False]
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

optimizer = GridSearchCV(RandomForestClassifier(), \
                         {'max_features' : [0.4, 0.6, 0.8, 1.0], 
                          'max_depth' : [5, 10, 15, 25, 35, 45, 55, 65], 
                          'n_estimators' : [1, 5, 10, 15, 30, 50, 70, 90, 100]},\
                         cv = 15, \
                         n_jobs=-1)
optimizer.fit(DF_train[['Age','Fare','Parch','Pclass','Title','CabinSymbol','Sex']], DF_train['Survived'])
print('Random Forest Best Result', optimizer.best_score_)
print('Best Parameters', optimizer.best_params_)

Random Forest Best Result 0.8462401795735129
Best Parameters {'max_depth': 10, 'max_features': 0.6, 'n_estimators': 90}


In [200]:
def writeAnswer(filename, optimizer):
    prediction = optimizer.predict(np.hstack([DF_test[['Age','Fare','Parch','Pclass','Title','CabinSymbol','Sex']]]))
    submission = pd.DataFrame({
            "PassengerId": test["PassengerId"],
            "Survived": prediction
        })
    submission.Survived=submission.Survived.astype('int64',copy=False)
    submission.to_csv(filename, index=False)
    
writeAnswer('RandomForestClassifier.csv', optimizer)

In [194]:
DF

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,Fulltitle,CabinSymbol,CabinNumber
0,22.000000,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,2,Mr.,8,NaN
1,38.000000,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,3,Mrs.,2,85.0
2,26.000000,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,0,1.0,STON/O2. 3101282,1,Miss.,8,NaN
3,35.000000,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,0,1,1.0,113803,3,Mrs.,2,123.0
4,35.000000,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,1,0,0.0,373450,2,Mr.,8,NaN
5,24.816367,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,1,0,0.0,330877,2,Mr.,8,NaN
6,54.000000,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,1,0,0.0,17463,2,Mr.,4,46.0
7,2.000000,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,1,3,0.0,349909,0,Master.,8,NaN
8,27.000000,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,0,0,1.0,347742,3,Mrs.,8,NaN
9,14.000000,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,0,1,1.0,237736,3,Mrs.,8,NaN
